<h2> Session 5.1 Inverse kinematics in CoppeliaSim with suction cup type manipulator</h2>

<br>- It requires that the files 'sim.py', 'simConst.py', 'remoteapi.dll' are hosted in the same folder as this Jupyter notebook. It also requires using the attached scene: RobotScaraManIK.ttt where a Scara robot appears, cylindrical primitives and a pneumatic actuator (suction cup) preprogrammed to pick up and down elements.

<br>- You have in the folder attached to the session these files ( guideactivitys5.zip )

#### You can follow this video to perform the guided part of the exercise:


In [24]:
from IPython.display import YouTubeVideo

YouTubeVideo('RmZZ2ZXt43U')

<br><h4> Remember:</h4>
<br>1. Open the scene RobotScaraManIK.ttt
<br>2. Check that in the scene hierarchy there are several scripts: in the base to connect to the Jupyter and in the Suction Cup (SuctionPad) there are two scripts to force from Jupyter its activation / deactivation. 
<br>3.Remember that the Coppelia scene must be in PLAY to be controlled from Jupyter.


<br><h4> Important notes: </h4>

1 Remember that the dynamic behavior of Joint 1 and 2 of the Scara del Coppelia ranges from -180º to 180º. And that in the Coppelia program you can see the data in degrees but that from the Jupyter you have to send the data in Radians.

2 Remember that the code can be improved by reducing the accuracy of nsolve and give a warning if there is no solution 



In [2]:
# We import the necessary libraries
import sim           # library to connect with CoppeliaSim
import sympy as sp   # library for symbolic calculation
import numpy as np
from sympy import *

In [3]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)

In [4]:
from sympy.physics.mechanics import dynamicsymbols
theta1, theta2, d3, lc, la, lb, theta, alpha, a, d = dynamicsymbols('theta1 theta2 d3 lc la lb theta alpha a d')
theta1, theta2, d3, lc, la, lb, theta, alpha, a, d 

(theta1, theta2, d3, lc, la, lb, theta, alpha, a, d)

### 0. We establish the connection
We will use the functions of the COPPELIA Remote API.

In [5]:
def connect(port):
# returns the client number or -1 if it cannot connect
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Connect
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID

<h3> 1. Suction cup control </h3>
Function that allows you to activate or deactivate the suction cup of the Coppelia from Jupyter. There are instructions in the Coppelia suctionPad scripts that allow remote use. 

In [6]:
def setEffector(val):
# function that triggers the end effector remotely
# val is Int with value 0 or 1 to disable or activate the final actuator.
    res,retInts,retFloats,retStrings,retBuffer=sim.simxCallScriptFunction(clientID,
        "suctionPad", sim.sim_scripttype_childscript,"setEffector",[val],[],[],"", sim.simx_opmode_blocking)
    return res

<h3> 2. We get the handlers and take an object with the suction cup</h3>


In [7]:
# We require the handlers for the joints, the suction cup and the suction sensor (Allows to know if the object is nearby)
clientID = connect(19999)

retCode,tip=sim.simxGetObjectHandle(clientID,'suctionPadSensor',sim.simx_opmode_blocking)
retCode,suction=sim.simxGetObjectHandle(clientID,'suctionPad',sim.simx_opmode_blocking)
retCode,joint1=sim.simxGetObjectHandle(clientID,'Joint1',sim.simx_opmode_blocking)
retCode,joint2=sim.simxGetObjectHandle(clientID,'Joint2',sim.simx_opmode_blocking)
retCode,joint3=sim.simxGetObjectHandle(clientID,'Joint3',sim.simx_opmode_blocking)
print(tip, suction, joint1, joint2, joint3)

conectado a 19999
24 22 17 19 21


In [9]:
# We deactivate the final actuator (suction cup)
setEffector(0)

0

In [10]:
# We send the positions to the joints, the first two in radians

q = [0, 0, 0.13] #posición Home and Collected Prism (Suction Cup Up)

retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_oneshot)


In [11]:
# We lower the suction cup to the object to be taken, without causing a collision (we place it aprox 1 mm above)

q = [0, 0, -0.089] 

retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_oneshot)

In [12]:
# We activate the final actuator (suction cup), it has to suck the object
setEffector(1)

0

In [13]:
# We move the Object to a position, remember that it is in Radians
q = [0.3, 0.3, 0.13]

retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_oneshot)

In [14]:
# we deactivate the final actuator (suction cup), we drop the piece
setEffector(0)

0

In [15]:
# y verificamos la posición del actuador
retCode,pos=sim.simxGetObjectPosition(clientID, tip, -1, sim.simx_opmode_blocking)
print(pos)

[0.35615992546081543, 0.17195305228233337, 0.1369566023349762]


<h3> 3. We prepare the Inverse Kinematics equations of our robot model. This was done in the previous session</h3>
Remember this Translation Matrix:<br>
<img src="ScaraManIK.png" width="500"><br>

In [151]:
# we prepare the equations by transforming the expressions
# from the form <expression = value> to the form <expression - value> = 0

In [16]:
# We define a destination point of the red piece (0.3,0.1, -0.089)
x = 0.21
y = 0.21
z = 0.02 #Esta is the height at which we have to take the piece

In [17]:
# eq1 is the one that gives us the value of the position X (eq=f(x)-x=0)

eq1 = 0.2 * cos(theta1) + 0.2 * cos(theta1 + theta2) - x
eq1

0.2*cos(theta1 + theta2) + 0.2*cos(theta1) - 0.21

In [18]:
# eq2 is the one that gives us the value of the position Y (eq=f(y)-y=0)
eq2 = (0.2 * sin(theta1) + 0.2 * sin(theta1 + theta2)) - y
eq2

0.2*sin(theta1 + theta2) + 0.2*sin(theta1) - 0.21

In [19]:
# eq3 is the one that gives us the value of the position Z (eq=f(z)-z=0)   0.108 is the real height of manipulator
eq3 = 0.108 - d3 - z
eq3

0.088 - d3

In [20]:
# Use the nsolve function to solve the matrices (remember that it gives the result in radians)
q=nsolve((eq1,eq2,eq3),(theta1,theta2,d3),(1,1,1))
print(q)

Matrix([[0.0513401678359322], [1.46811599112303], [0.0880000000000000]])


<h3> 4. We send previously calculated angles to each axis</h3>

In [21]:
retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)


<h3> We can check in Coppelia that it is located just above the red object <br>
<img src="red.png" width="500">

In [22]:
# Now we can go down to pick up the object. You have to take it approx 1mm above its surface
retCode = sim.simxSetJointTargetPosition(clientID, joint3, -q[2], sim.simx_opmode_oneshot)

In [23]:
# Now I buid a seguence of movements
import time
setEffector(1)
time.sleep(1)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0.13, sim.simx_opmode_oneshot)
time.sleep(1)
retCode = sim.simxSetJointTargetPosition(clientID, joint1, 0, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, 0, sim.simx_opmode_oneshot)
time.sleep(1)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, -0.088, sim.simx_opmode_oneshot)
time.sleep(1)
setEffector(0)
time.sleep(1)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0.13, sim.simx_opmode_oneshot)


<h3> 5. Build a sequence of movements </h3>

#### Using the procedure for calculating inverse kinematics and the time library, try to place the three existing objects on top of each other.
<br>- Note1: we can get the position of each object with 
<br> retCode,pos=sim.simxGetObjectPosition(clientID, nombre_del_objeto, -1, sim.simx_opmode_blocking)
<br> print(pos)
